In [1]:
!nvidia-smi

Fri Nov 15 15:45:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [27]:
import pickle
import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
from datetime import datetime
import os
import pandas as pd
from google.colab import files




In [14]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [15]:
# Define the custom model class (the one you used when training the model)
class EmotionRecognitionModel(nn.Module):
    def __init__(self, num_classes):
        super(EmotionRecognitionModel, self).__init__()
        self.features = models.resnet18(pretrained=False)  # Custom training, no pretraining
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        return self.features(x)

# Load the model (ensure it's loaded on the specified device)
model_path = "/content/emotion_detection_model.pkl"
with open(model_path, 'rb') as f:
    model = pickle.load(f)

In [16]:
model_path = "/content/emotion_detection_model.pkl"
with open(model_path, 'rb') as f:
    model = pickle.load(f)

# Set the model to evaluation mode and move it to the GPU
model.eval()
model.to(device)

EmotionRecognitionModel(
  (features): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [18]:

# Load label encoder from the label_encoding.pkl file
label_encoding_path = "/content/label_encoder.pkl"
with open(label_encoding_path, 'rb') as f:
    label_encoder = pickle.load(f)


In [20]:
# Define image transformations (resize and normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [25]:
sample_image_path = "/content/S046_003_00000016.png"  # Change to a real image path
image = Image.open(sample_image_path).convert('RGB')

# Apply the necessary transformations to the image
image = transform(image).unsqueeze(0)  # Add batch dimension

# Move the image to the GPU
image = image.to(device)


In [26]:
# Run the model on the image
with torch.no_grad():
    output = model(image)

# Get the predicted class
_, predicted_class = torch.max(output, 1)

# Map the predicted class back to the original label using the label encoder
predicted_label = label_encoder.inverse_transform([predicted_class.item()])

# Print the prediction result
print(f"Predicted label: {predicted_label[0]}")

Predicted label: sad


In [28]:
# Time check function (Working hours 9:30 AM to 10:00 AM)
def is_within_working_time():
    current_time = datetime.now().time()
    start_time = datetime.strptime("09:30:00", "%H:%M:%S").time()
    end_time = datetime.strptime("10:00:00", "%H:%M:%S").time()
    return start_time <= current_time <= end_time



In [29]:
def predict_emotion(image):
    if not is_within_working_time():
        print("Model can only work between 9:30 AM and 10:00 AM.")
        return

    # Load and preprocess image
    image = Image.open(image).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    # Get model predictions
    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)
        predicted_emotion = label_encoder.inverse_transform([predicted_class.item()])[0]

    # Get current timestamp
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Save prediction to CSV
    results = {
        "image_path": str(image),
        "predicted_emotion": predicted_emotion,
        "timestamp": timestamp
    }

    # Check if the CSV file exists
    csv_file = "/content/emotion_predictions.csv"
    if os.path.exists(csv_file):
        # Append to CSV if file exists
        df = pd.read_csv(csv_file)
        df = df.append(results, ignore_index=True)
    else:
        # Create a new CSV file if it doesn't exist
        df = pd.DataFrame([results])

    # Save the data to CSV
    df.to_csv(csv_file, index=False)

    print(f"Prediction: {predicted_emotion}")
    print(f"Timestamp: {timestamp}")
    print("Prediction saved to CSV.")

    # Provide download link for CSV
    files.download(csv_file)

In [30]:
# Upload image and make prediction
from google.colab import files
uploaded = files.upload()

for image_name in uploaded.keys():
    print(f"Making prediction for {image_name}...")
    predict_emotion(image_name)

Saving S011_003_00000013.png to S011_003_00000013.png
Making prediction for S011_003_00000013.png...
Model can only work between 9:30 AM and 10:00 AM.
